# Day 16

In [23]:
import requests
from bs4 import BeautifulSoup

def get_aoc_problem(day, year=2023):
    url = f"https://adventofcode.com/{year}/day/{day}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # raises an exception for HTTP errors

        soup = BeautifulSoup(response.text, 'html.parser')
        
        problem_text = soup.find('article').get_text()
        return problem_text
    except Exception as e:
        return f"Error fetching problem: {e}"

day = 16
problem_prompt = get_aoc_problem(day)
print(problem_prompt)

--- Day 16: The Floor Will Be Lava ---With the beam of light completely focused somewhere, the reindeer leads you deeper still into the Lava Production Facility. At some point, you realize that the steel facility walls have been replaced with cave, and the doorways are just cave, and the floor is cave, and you're pretty sure this is actually just a giant cave.
Finally, as you approach what must be the heart of the mountain, you see a bright light in a cavern up ahead. There, you discover that the beam of light you so carefully focused is emerging from the cavern wall closest to the facility and pouring all of its energy into a contraption on the opposite side.
Upon closer inspection, the contraption appears to be a flat, two-dimensional square grid containing empty space (.), mirrors (/ and \), and splitters (| and -).
The contraption is aligned so that most of the beam bounces around the grid, but each tile on the grid converts some of the beam's light into heat to melt the rock in th

In [57]:
try:
    # Open and read the file
    with open('input.txt', 'r') as file:
        lines = file.read().strip().split('\n')

    # Print each line
    for line in lines:
        print(line)

except FileNotFoundError:
    # Specific exception for a clearer error message
    print('Input file not found.')

except Exception as e:
    # Catch other exceptions and print the error
    print(f'An error occurred: {e}')


\...\............................../...\....-..../...|....../.......-...........................|.............
.............|/.../............................-....|......../...........|.|................/\-..|....\...../.
....\......-.....||-\........................|/........|............/.......................|\...|............
|.....-....\.|......-.\....\-......|.........../|......../\...\.-..........-.........\...-..\........./.......
............../.|..........|.......|.......\.\...............|../......................\......................
.-.-...|....-.-.......\...\..../....../.......-..................\......./....................-........-......
..........\.....|.....\.....\..................................................................|..............
...../..................../...........-..-.........................-...................................../-...
......\......\.............|..../..........\.........................\..\../....../...........-...........-...
.

### Utility functions

In [25]:
import numpy as np

def calc_new_pos(curr, direction, debug=False):
    """ Calculate the new position and directions after 1 step """

    prev_dirs[curr].append(direction)

    output_list = []

    if grid[(curr[0],curr[1])] == '.':
        if debug:
            print(f'{curr} encountered empty space : keep moving')
        new_position = tuple(np.add(np.array(curr),np.array(direction)))
        output_list.append((new_position,direction))

    elif grid[(curr[0],curr[1])] == '/':

        # LEFT = (-1, 0)
        # UP = (0, -1)
        # RIGHT = (1, 0)
        # DOWN = (0, 1)

        if direction == (1,0):
            # RIGHT -> UP
            new_direction = (0,-1)
        elif direction == (0,-1):
            # UP -> RIGHT
            new_direction = (1,0)
        elif direction == (-1,0):
            new_direction = (0,1)
            # LEFT -> DOWN
        elif direction == (0,1):
            # DOWN -> LEFT
            new_direction = (-1,0)
        else:
            print('Error occured')

        new_position = tuple(np.add(np.array(curr),np.array(new_direction)))
        output_list.append((new_position,new_direction))
        
    elif grid[(curr[0],curr[1])] == '\\':
        # print(f'{curr} encountered a mirror : changing direction')

        # LEFT = (-1, 0)
        # UP = (0, -1)
        # RIGHT = (1, 0)
        # DOWN = (0, 1)

        if direction == (1,0):
            # RIGHT -> DOWN
            new_direction = (0,1)
        elif direction == (0,1):
            # DOWN -> RIGHT
            new_direction = (1,0)
        elif direction == (-1,0):
            new_direction = (0,-1)
            # LEFT -> UP
        elif direction == (0,-1):
            # UP -> LEFT
            new_direction = (-1,0)
        else:
            print('Error occured')

        new_position = tuple(np.add(np.array(curr),np.array(new_direction)))
        output_list.append((new_position,new_direction))
        
    elif grid[(curr[0],curr[1])] == '-':
        if debug:
            print(f'{curr} encountered a - splitter : splitting direction')

        if direction[1] == 0:
            if debug:
                print('same direction : passing straight through..')
            new_position = tuple(np.add(np.array(curr),np.array(direction)))
            output_list.append((new_position,direction))
        else:
            if debug:
                print('opposite direction : (splitting left and right)')
            
            # left
            new_left_direction = np.array((-1, 0))
            new_left_pos = tuple(np.add(np.array(curr), new_left_direction))
            output_list.append((new_left_pos,(-1, 0)))
            
            # right
            new_right_direction = np.array((1, 0))
            new_right_pos = new_left_pos = tuple(np.add(np.array(curr), new_right_direction))
            output_list.append((new_right_pos,(1, 0)))
        
    elif grid[(curr[0],curr[1])] == '|':
        if debug:
            print(f'{curr} encountered a | splitter : splitting direction')
        
        if direction[0] == 0:
            if debug:
                print('same direction : passing straight through..')
            new_position = tuple(np.add(np.array(curr),np.array(direction)))
            output_list.append((new_position,direction))
        else:
            if debug:
                print('opposite direction : (splitting up and down)')
            
            # up
            new_up_direction = np.array((0, -1))
            new_up_pos = tuple(np.add(np.array(curr), new_up_direction))
            output_list.append((new_up_pos,(0, -1)))
            
            # down
            new_down_direction = np.array((0, 1))
            new_down_pos = tuple(np.add(np.array(curr), new_down_direction))
            output_list.append((new_down_pos,(0, 1)))

    else:
        print('Something weird happened...') 

    for item in output_list:
        assert type(item) == tuple
        assert type(item[0]) == tuple
        assert type(item[1]) == tuple
        
    return output_list

In [26]:
def check_cell(position):
    '''
    return True if position is within the grid
    else False
    '''
    if ((0 <= position[0][0] < len(lines[0])) and (0 <= position[0][1] < len(lines))):
        return True
    else:
        return False

### Part 1

In [27]:
current_cells = [(0,0)] # starting position
current_dirs = [(1,0)] # starting by moving right
next_move_stack = [(current_cells[0], current_dirs[0])]
counter = 0

# set up dictionary of symbols

grid = {}
prev_dirs = {}

for j in range(len(lines)):
    for i in range(len(lines[0])):
        grid[(i,j)] = lines[j][i]
        prev_dirs[(i,j)] = []

while next_move_stack != [] and counter < 1000:

    assert len(current_cells) == len(current_dirs)

    # execute a move for each current

    for move in next_move_stack:
        # move = (current_cell, current_dir)
        # current_cell = (x,y) x = col, y = row

        # LEFT = (-1, 0)
        # UP = (0, -1)
        # RIGHT = (1, 0)
        # LEFT = (0, 1)

        next_move_stack.remove(move)

        if move[1] in prev_dirs[move[0]]:
            # already moved here
            continue
        else:
            new_positions = calc_new_pos(move[0], move[1])
            valid_new_positions = filter(check_cell, new_positions)
            if valid_new_positions:
                next_move_stack.extend(valid_new_positions)

    # print(f'counter = {counter}')        
    # print(f'next_move_stack = {next_move_stack}')        

    counter += 1

if counter == 1000:
    print('STOPPED DUE TO COUNTER')


In [28]:
energy_count = 0

for key in prev_dirs.keys():
    if prev_dirs[key] != []:
        energy_count += 1


print(f'Total energized cells = {energy_count}')

Total energized cells = 6795


### Part 2

In [47]:
def calc_new_pos_2(curr, direction, prev_dirs, debug=False):
    """ Calculate the new position and directions after 1 step """

    prev_dirs[curr].append(direction)

    output_list = []

    if grid[(curr[0],curr[1])] == '.':
        if debug:
            print(f'{curr} encountered empty space : keep moving')
        new_position = tuple(np.add(np.array(curr),np.array(direction)))
        output_list.append((new_position,direction))

    elif grid[(curr[0],curr[1])] == '/':

        # LEFT = (-1, 0)
        # UP = (0, -1)
        # RIGHT = (1, 0)
        # DOWN = (0, 1)

        if direction == (1,0):
            # RIGHT -> UP
            new_direction = (0,-1)
        elif direction == (0,-1):
            # UP -> RIGHT
            new_direction = (1,0)
        elif direction == (-1,0):
            new_direction = (0,1)
            # LEFT -> DOWN
        elif direction == (0,1):
            # DOWN -> LEFT
            new_direction = (-1,0)
        else:
            print('Error occured')

        new_position = tuple(np.add(np.array(curr),np.array(new_direction)))
        output_list.append((new_position,new_direction))
        
    elif grid[(curr[0],curr[1])] == '\\':
        # print(f'{curr} encountered a mirror : changing direction')

        # LEFT = (-1, 0)
        # UP = (0, -1)
        # RIGHT = (1, 0)
        # DOWN = (0, 1)

        if direction == (1,0):
            # RIGHT -> DOWN
            new_direction = (0,1)
        elif direction == (0,1):
            # DOWN -> RIGHT
            new_direction = (1,0)
        elif direction == (-1,0):
            new_direction = (0,-1)
            # LEFT -> UP
        elif direction == (0,-1):
            # UP -> LEFT
            new_direction = (-1,0)
        else:
            print('Error occured')

        new_position = tuple(np.add(np.array(curr),np.array(new_direction)))
        output_list.append((new_position,new_direction))
        
    elif grid[(curr[0],curr[1])] == '-':
        if debug:
            print(f'{curr} encountered a - splitter : splitting direction')

        if direction[1] == 0:
            if debug:
                print('same direction : passing straight through..')
            new_position = tuple(np.add(np.array(curr),np.array(direction)))
            output_list.append((new_position,direction))
        else:
            if debug:
                print('opposite direction : (splitting left and right)')
            
            # left
            new_left_direction = np.array((-1, 0))
            new_left_pos = tuple(np.add(np.array(curr), new_left_direction))
            output_list.append((new_left_pos,(-1, 0)))
            
            # right
            new_right_direction = np.array((1, 0))
            new_right_pos = new_left_pos = tuple(np.add(np.array(curr), new_right_direction))
            output_list.append((new_right_pos,(1, 0)))
        
    elif grid[(curr[0],curr[1])] == '|':
        if debug:
            print(f'{curr} encountered a | splitter : splitting direction')
        
        if direction[0] == 0:
            if debug:
                print('same direction : passing straight through..')
            new_position = tuple(np.add(np.array(curr),np.array(direction)))
            output_list.append((new_position,direction))
        else:
            if debug:
                print('opposite direction : (splitting up and down)')
            
            # up
            new_up_direction = np.array((0, -1))
            new_up_pos = tuple(np.add(np.array(curr), new_up_direction))
            output_list.append((new_up_pos,(0, -1)))
            
            # down
            new_down_direction = np.array((0, 1))
            new_down_pos = tuple(np.add(np.array(curr), new_down_direction))
            output_list.append((new_down_pos,(0, 1)))

    else:
        print('Something weird happened...') 

    for item in output_list:
        assert type(item) == tuple
        assert type(item[0]) == tuple
        assert type(item[1]) == tuple
        
    return output_list, prev_dirs

In [55]:
def energy_calculator_function(starting_cell, starting_direction):
    
    next_move_stack = [(starting_cell, starting_direction)]
    counter = 0

    # set up dictionary of symbols

    prev_dirs = {}

    for j in range(len(lines)):
        for i in range(len(lines[0])):
            prev_dirs[(i,j)] = []

    while next_move_stack != [] and counter < 1000:

        # print(f'next_move_stack = {next_move_stack}')

        assert len(current_cells) == len(current_dirs)

        # execute a move for each current

        for move in next_move_stack:
            # move = (current_cell, current_dir)
            # current_cell = (x,y) x = col, y = row

            # LEFT = (-1, 0)
            # UP = (0, -1)
            # RIGHT = (1, 0)
            # LEFT = (0, 1)

            next_move_stack.remove(move)

            if move[1] in prev_dirs[move[0]]:
                # already moved here
                continue
            else:
                new_positions, prev_dirs = calc_new_pos_2(move[0], move[1], prev_dirs)
                valid_new_positions = filter(check_cell, new_positions)
                if valid_new_positions:
                    next_move_stack.extend(valid_new_positions)

        # print(f'counter = {counter}')        
        # print(f'next_move_stack = {next_move_stack}')        

        counter += 1

    if counter == 1000:
        print('STOPPED DUE TO COUNTER')

    energy_count = 0

    for key in prev_dirs.keys():
        if prev_dirs[key] != []:
            energy_count += 1


    # print(f'Total energized cells = {energy_count}')
    # print(f'returning {energy_count}')
    return energy_count

In [58]:
grid = {}

for j in range(len(lines)):
    for i in range(len(lines[0])):
        grid[(i,j)] = lines[j][i]

max_energy_count = 0

# top : 1 <= i < width, j = 0
for i in range(1, len(lines[0])):
    energy_count = energy_calculator_function((i,0), (0,1))
    print(f'(i,j) = {(i,0)}, energy = {energy_count}')

    if energy_count > max_energy_count:
        max_energy_count = energy_count

# right : i = -1, 1 <= j < height
for j in range(1, len(lines)):
    energy_count = energy_calculator_function((len(lines[0])-1,j), (-1,0))
    print(f'(i,j) = {(len(lines[0])-1,j)}, energy = {energy_count}')
    
    if energy_count > max_energy_count:
        max_energy_count = energy_count

# bottom : 1 <= i < width, j = -1 
for i in range(1, len(lines[0])):
    energy_count = energy_calculator_function((i,len(lines)-1), (0,-1))
    print(f'(i,j) = {(i,len(lines)-1)}, energy = {energy_count}')
    
    if energy_count > max_energy_count:
        max_energy_count = energy_count

# left : i = 0, 1 <= j < height
for j in range(1, len(lines)):
    energy_count = energy_calculator_function((0,j), (1,0))
    print(f'(i,j) = {(0,j)}, energy = {energy_count}')
    
    if energy_count > max_energy_count:
        max_energy_count = energy_count


# extra corners
extra_corners = [
    # top left : going down
    ((0,0),(0,1)),
    # top right : going left
    ((len(lines[0])-1,0),(-1,0)),
    # bottom right : going up
    ((len(lines[0])-1,len(lines)-1),(0,-1)),
    # bottom left : going right
    ((0,len(lines)-1),(1,0))
]
for move in extra_corners:
    energy_count = energy_calculator_function(move[0], move[1])
    if energy_count > max_energy_count:
        max_energy_count = energy_count

print(max_energy_count)

(i,j) = (1, 0), energy = 143
(i,j) = (2, 0), energy = 6962
(i,j) = (3, 0), energy = 143
(i,j) = (4, 0), energy = 32
(i,j) = (5, 0), energy = 13
(i,j) = (6, 0), energy = 6798
(i,j) = (7, 0), energy = 131
(i,j) = (8, 0), energy = 6845
(i,j) = (9, 0), energy = 6953
(i,j) = (10, 0), energy = 6803
(i,j) = (11, 0), energy = 54
(i,j) = (12, 0), energy = 6865
(i,j) = (13, 0), energy = 79
(i,j) = (14, 0), energy = 81
(i,j) = (15, 0), energy = 6830
(i,j) = (16, 0), energy = 6795
(i,j) = (17, 0), energy = 33
(i,j) = (18, 0), energy = 23
(i,j) = (19, 0), energy = 7154
(i,j) = (20, 0), energy = 6907
(i,j) = (21, 0), energy = 6816
(i,j) = (22, 0), energy = 58
(i,j) = (23, 0), energy = 6866
(i,j) = (24, 0), energy = 58
(i,j) = (25, 0), energy = 6823
(i,j) = (26, 0), energy = 16
(i,j) = (27, 0), energy = 6803
(i,j) = (28, 0), energy = 6806
(i,j) = (29, 0), energy = 6812
(i,j) = (30, 0), energy = 6795
(i,j) = (31, 0), energy = 16
(i,j) = (32, 0), energy = 71
(i,j) = (33, 0), energy = 111
(i,j) = (34, 0